In [1]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
resolution = 10

eps_si = 3.54**2
eps_pdms = 1.4**2

sx = 3
sy = 3
sz = 4

dpml = 1


cell = mp.Vector3(sx, sy, sz)
blk = mp.Block(size = mp.Vector3(sx, sy, mp.inf),
               center = mp.Vector3(),
               material = mp.Medium(epsilon = eps_pdms)
              )
geometry = [blk]

pml_layers = [mp.PML(dpml)]
fcen = 1.5
df = 0.25
src = [mp.Source(src = mp.GaussianSource(fcen, fwidth=df),
                component = mp.Ez,
                center = mp.Vector3(0,0,-(sz/2 - dpml - 0.5)),
                size = mp.Vector3(sx,sy,0)
               )]
sim = mp.Simulation(cell_size=cell,
                    geometry=geometry,
                    boundary_layers=pml_layers,
                    sources=src,
                    resolution = resolution
                   )

In [ ]:
freg = mp.FluxRegion(center=mp.Vector3(0,0,sz/2-dpml-0.5),
                     size = mp.Vector3(sx,sy,0)
                    )
nfreq = 500
trans = sim.add_flux(fcen, df, nfreq, freg)
pt = mp.Vector3(0,0,sz/2-dpml-0.75)
sim.run(until_after_sources = mp.stop_when_fields_decayed(50, mp.Ex, pt,1e-3))

-----------
Initializing structure...
field decay(t = 50.050000000000004): 7.904024014929061e-35 / 7.904024014929061e-35 = 1.0
field decay(t = 100.05000000000001): 5.801575334453588e-37 / 7.904024014929061e-35 = 0.00734002746385337
field decay(t = 150.1): 4.21142120021762e-37 / 7.904024014929061e-35 = 0.005328198892441519
field decay(t = 200.15): 6.262987359765109e-37 / 7.904024014929061e-35 = 0.007923795965112992
field decay(t = 250.20000000000002): 9.240465269355892e-37 / 7.904024014929061e-35 = 0.011690836530737471
field decay(t = 300.2): 1.2463006880626981e-36 / 7.904024014929061e-35 = 0.01576792638418981
field decay(t = 350.25): 1.5439710937344562e-36 / 7.904024014929061e-35 = 0.019533987887919053
field decay(t = 400.3): 1.7917816896359236e-36 / 7.904024014929061e-35 = 0.022669233876967225
field decay(t = 450.35): 1.985994176966234e-36 / 7.904024014929061e-35 = 0.02512636820453864
field decay(t = 500.40000000000003): 2.1345114796925352e-36 / 7.904024014929061e-35 = 0.0270053769530

In [ ]:
straight = mp.get_fluxes(trans)

In [ ]:
straight

In [ ]:
sim.reset_meep()
geometry.append(mp.Cylinder(radius = 1, height = 1)
               ,center = mp.Vector3())
sim = mp.Simulation(cell_size=cell,
                    geometry=geometry,
                    boundary_layers=pml_layers,
                    sources=src,
                    resolution = resolution
                   )
freg = mp.FluxRegion(center=mp.Vector3(0,0,sz/2-dpml-0.5),
                     size = mp.Vector3(sx,sy,0)
                    )
nfreq = 500
trans = sim.add_flux(fcen, df, nfreq, freg)
pt = mp.Vector3(0,0,sz/2-dpml-0.75)
sim.run(until_after_sources = mp.stop_when_fields_decayed(50, mp.Ex, pt,1e-3))

In [ ]:
bend = mp.get_fluxes(trans)
flux_freqs = mp.get_flux_freqs(trans)
Ts = []
wl = []
for i in range(nfreq):
    wl = np.append(wl, 1/flux_freqs[i])
    Ts = np.append(Ts, bend[i]/straight[i])
plt.figure()
plt.plot(wl,Ts)